In [1]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/20_0.py
points=[[0.7200384213053443, 0.7684801288877052], [0.11814864763102939, 0.4914770281289862], [0.9212214108948671, 0.9306800954633722], [0.4249951441909722, 0.8559799374904841], [0.04664940393785011, 0.012985197746691601], [0.40929113837661246, 0.9581483142120293], [0.1826115009373238, 0.4878700875871902], [0.49994276239008717, 0.621414790704136], [0.7199490776824592, 0.175295512420384], [0.11333428261361567, 0.2610054569554987], [0.7242990587725784, 0.737945943215749], [0.3594088921425578, 0.569464241395479], [0.8942499564724715, 0.45769898100900486], [0.614480546062911, 0.9702498379178963], [0.30328164543483094, 0.27966226444785547], [0.7329862630767433, 0.9895797595284683], [0.8471894346272434, 0.31804973459786534], [0.06820018184617793, 0.31200200468111094], [0.46790814492463284, 0.4315543450694791], [0.37617923758954075, 0.5298501020240104]]

In [2]:
import gurobipy as gp
from gurobipy import GRB
from itertools import combinations
import math
#构造数据集
city=list(range(len(points)))
coordinates={} # dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
#计算距离
def distance(point1,point2):
    c1=coordinates[point1]
    c2=coordinates[point2]
    Differ=(c1[0]-c2[0],c1[1]-c2[1])
    return math.sqrt(Differ[0]*Differ[0]+Differ[1]*Differ[1])

dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)}
#建立模型
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') #binary表示变量取值为0,1
for i,j in vars.keys():
    vars[j,i]=vars[i,j]## 第一个约束, i.e. x_ij=x_ji
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city) ## 第二个约束:sum_(i,j)x_ij=2 forall i    
#计算最短的子环
def subtour(edges):
    unvisited=city[:]
    tour=city[:]
    while unvisited:
        curtour=[]
        neighbors=unvisited
        while neighbors:
            current=neighbors[0]
            curtour.append(current)
            unvisited.remove(current)
            neighbors=[j for i, j in edges.select(current,'*') if j in unvisited] 
        if len(curtour)<=len(tour):
            tour=curtour ## 用更短的子环替换
    return tour
#使用lazy-cut消除子环
def subtoureli(model,where):
    if where==GRB.Callback.MIPSOL:
        vals=model.cbGetSolution(model._vars)
        selected=gp.tuplelist((i,j) for i,j in model._vars.keys() if vals[i,j]>0.5) 
        tour=subtour(selected)                                                       
        if len(tour)<len(city):# 说明它是一个需要被消除的子环
            model.cbLazy(gp.quicksum(model._vars[i,j] for i,j in combinations(tour,2))<=len(tour)-1)
# 回到TSP问题
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-05-02
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 20 rows, 190 columns and 380 nonzeros
Model fingerprint: 0xc23c5899
Variable types: 0 continuous, 190 integer (190 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.02s
Presolved: 20 rows, 190 columns, 380 nonzeros
Variable types: 0 continuous, 190 integer (190 binary)

Root relaxation: objective 3.701342e+00, 33 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       3.7483064    3.74831  0.00%     -  

In [3]:
#我们现在给出最短的子环
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour #输出最优解

[0, 2, 15, 13, 5, 3, 7, 11, 19, 6, 1, 17, 9, 4, 14, 18, 8, 16, 12, 10]

In [4]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/20_1.py
points=[[0.4624399480492364, 0.17266962629344207], [0.4950119152405682, 0.8956002201877953], [0.9536457121086541, 0.5273810656216931], [0.6679230430036367, 0.91342796832032], [0.16514796660137931, 0.05780482986582702], [0.41000018801658955, 0.3507586254937931], [0.900636187342358, 0.5098462930600423], [0.28391002429289647, 0.12382152393783297], [0.605610117196593, 0.38025949910126566], [0.8547517884512781, 0.7786031740357802], [0.7685609251862037, 0.06807029265407694], [0.7656617351078263, 0.5207542804755099], [0.0951582537162089, 0.036079388369921195], [0.915561849563609, 0.82133147195706], [0.4300957871596124, 0.8812358410219993], [0.9672155298960079, 0.8044838025865045], [0.8145220686777632, 0.7000340654766124], [0.4346120149921042, 0.45692932221757254], [0.7783688605693182, 0.5611167464823495], [0.7584611451617201, 0.2673329671381115]]

In [5]:
city=list(range(len(points)))
coordinates={} ##dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)} 
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') 
for i,j in vars.keys():
    vars[j,i]=vars[i,j]
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city)
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 20 rows, 190 columns and 380 nonzeros
Model fingerprint: 0xd2f80701
Variable types: 0 continuous, 190 integer (190 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 20 rows, 190 columns, 380 nonzeros
Variable types: 0 continuous, 190 integer (190 binary)

Root relaxation: objective 3.143127e+00, 27 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.40619    0   10          -    3.40619      -     -    0s
*    0     0               0       3.5137257    3.51373  0.00%     -    0s

Cutting pl

[0, 7, 4, 12, 5, 17, 14, 1, 3, 13, 15, 9, 16, 2, 6, 18, 11, 8, 19, 10]

In [6]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/20_2.py
points=[[0.3634067295197678, 0.041104704136583625], [0.11036421408977559, 0.14179145472471233], [0.5127607284944661, 0.7488840601045168], [0.8367839382493641, 0.5998488182691877], [0.33970618136747543, 0.7176863557502245], [0.4434417335425458, 0.6830548782979923], [0.560107229128069, 0.44034295711596894], [0.7245227955338298, 0.739074719404094], [0.1301768529917926, 0.6551631680306983], [0.641874660019984, 0.16028806359871073], [0.6686020286943352, 0.04144631348683936], [0.2131207528895096, 0.741585387685736], [0.09573463904462287, 0.25880361272898533], [0.8406934085319321, 0.07510714702838384], [0.3084961322827017, 0.8325238539668954], [0.11376262570384321, 0.9214134006525027], [0.6394297352645405, 0.8976361723299751], [0.1335758897144489, 0.5142429970222804], [0.04419146005020924, 0.13064016602331718], [0.49270743739497136, 0.0629587011987961]]

In [7]:
city=list(range(len(points)))
coordinates={} ##dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)} 
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') 
for i,j in vars.keys():
    vars[j,i]=vars[i,j]
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city)
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 20 rows, 190 columns and 380 nonzeros
Model fingerprint: 0x9138feae
Variable types: 0 continuous, 190 integer (190 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 20 rows, 190 columns, 380 nonzeros
Variable types: 0 continuous, 190 integer (190 binary)
Found heuristic solution: objective 3.6053089

Root relaxation: cutoff, 30 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0         3.60531    3.60531  0.00%     -    0s

Explored 1 nodes (30 simplex iterations) in 0.03 second

[0, 1, 18, 12, 17, 8, 11, 15, 14, 4, 5, 2, 16, 7, 3, 6, 9, 13, 10, 19]

In [8]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/40_0.py
points=[[0.8499424072568177, 0.37557049535188114], [0.3639425386961409, 0.5383351019619592], [0.388522461107798, 0.652255633279069], [0.920203915524144, 0.49918677380570187], [0.2088467943029596, 0.16856475462117004], [0.6130492911152661, 0.08441457036571576], [0.670093914533608, 0.027857373597625856], [0.8511419966420318, 0.32691280159311253], [0.32562990553968374, 0.6229690496897505], [0.011396479537148174, 0.8051964633478813], [0.320288341449639, 0.4399060357810438], [0.9361811560095145, 0.06146712607039795], [0.10371755605918354, 0.960371762112271], [0.23570850416193156, 0.15649358562497506], [0.48457707373231496, 0.8369440338139559], [0.025359066379691098, 0.49939043890831913], [0.023376823334245067, 0.7603815051664067], [0.27796931689591464, 0.4188724951099786], [0.19027664112296228, 0.6346243354490421], [0.6538684679036535, 0.8283860720970725], [0.8435158009824851, 0.418290178998864], [0.13171638382509432, 0.6145397467554525], [0.3143965150415994, 0.22379625156226268], [0.9084931814766005, 0.1105140512187075], [0.1416947837029746, 0.41303872334835623], [0.8509120581708949, 0.1568952744022627], [0.5304963747757846, 0.4140724976255843], [0.5031543679898768, 0.20372315896231374], [0.037428597136766384, 0.3496907877219887], [0.4053943344628794, 0.43723140091495516], [0.2643865387042419, 0.22411988825301132], [0.9115035986126351, 0.7694733835524424], [0.9470642525792283, 0.7934839525248202], [0.11102847515820624, 0.6678079933634947], [0.9755713227162448, 0.4432605749478816], [0.1481933445495498, 0.01853926893878155], [0.44693007215513303, 0.8773974627876209], [0.8553257874441131, 0.424096602767585], [0.8325599343256758, 0.9204372189058009], [0.7481858982466353, 0.2714758637255349]]

In [9]:
city=list(range(len(points)))
coordinates={} ##dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)} 
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') 
for i,j in vars.keys():
    vars[j,i]=vars[i,j]
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city)
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 40 rows, 780 columns and 1560 nonzeros
Model fingerprint: 0x689233e8
Variable types: 0 continuous, 780 integer (780 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 40 rows, 780 columns, 1560 nonzeros
Variable types: 0 continuous, 780 integer (780 binary)

Root relaxation: objective 4.574380e+00, 54 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.57438    0    8          -    4.57438      -     -    0s
     0     0    4.91948    0    6          -    4.91948      -     -    0s
*    0   

[0,
 7,
 39,
 25,
 23,
 11,
 6,
 5,
 27,
 22,
 30,
 13,
 4,
 35,
 28,
 15,
 24,
 17,
 10,
 29,
 26,
 1,
 2,
 8,
 18,
 21,
 33,
 16,
 9,
 12,
 36,
 14,
 19,
 38,
 32,
 31,
 3,
 34,
 37,
 20]

In [10]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/40_1.py
points=[[0.44892105995514564, 0.9630483751903149], [0.36870807933210836, 0.3308825422087206], [0.0019540470943286703, 0.6384323812025432], [0.008899431809812897, 0.41484715125483673], [0.3072394515248973, 0.9381541690197118], [0.4758880725150134, 0.8119225674037281], [0.30380871292562506, 0.10982077408557667], [0.7675811158451529, 0.8592091349416751], [0.7615417291170622, 0.6919182889162965], [0.4584965844683988, 0.9791886490411906], [0.037071611249517855, 0.13414880247388783], [0.19082786138339436, 0.9727496926100112], [0.7122940681331653, 0.4469657786204072], [0.8409772086100726, 0.5335793557608145], [0.2956001042745008, 0.3757705135122519], [0.22168571850876462, 0.5872520288858839], [0.7535773331581925, 0.5282340756736977], [0.7054577140064786, 0.5761869950708136], [0.7760415141128949, 0.4260665556510448], [0.30980575334077254, 0.30657782449644444], [0.2757568935071151, 0.1672346744185862], [0.4841611071779435, 0.9964948339602361], [0.2680363694049197, 0.7469076809767077], [0.3205576217193462, 0.20249084768472259], [0.21974364617694864, 0.20516988323111418], [0.8465817430899267, 0.04797279180700109], [0.0010476309926022997, 0.9436871464177482], [0.2301874169561352, 0.5136959316415929], [0.7092581373193645, 0.9591417723884094], [0.7280813513839333, 0.2776922778619012], [0.8589431688883274, 0.7964532922518944], [0.4790538096508661, 0.04262794243961621], [0.6550459342796415, 0.5294349342762747], [0.9222242424537025, 0.27748456835027835], [0.44914766159222175, 0.7957655116157663], [0.010810367701279522, 0.2560484466886048], [0.44092596148169283, 0.2250056781110329], [0.6025347671724595, 0.7190666430544067], [0.3991336213759181, 0.02464570689774892], [0.4404226537051551, 0.5704284381421915]]

In [11]:
city=list(range(len(points)))
coordinates={} ##dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)} 
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') 
for i,j in vars.keys():
    vars[j,i]=vars[i,j]
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city)
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 40 rows, 780 columns and 1560 nonzeros
Model fingerprint: 0x7fbda182
Variable types: 0 continuous, 780 integer (780 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 40 rows, 780 columns, 1560 nonzeros
Variable types: 0 continuous, 780 integer (780 binary)

Root relaxation: objective 5.318703e+00, 63 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.31870    0    6          -    5.31870      -     -    0s
H    0     0                       5.5740607    5.31870  4.58%     -    0s
     0   

[0,
 4,
 11,
 26,
 2,
 3,
 35,
 10,
 24,
 20,
 23,
 6,
 38,
 31,
 36,
 1,
 19,
 14,
 27,
 15,
 22,
 34,
 5,
 37,
 39,
 32,
 12,
 18,
 29,
 25,
 33,
 13,
 16,
 17,
 8,
 30,
 7,
 28,
 21,
 9]

In [12]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/40_2.py
points=[[0.3355900731321032, 0.6678754437049276], [0.05822626343056314, 0.17920312247865844], [0.67595647386004, 0.7005457134486452], [0.8272714286308914, 0.8746099881102454], [0.7990763480315627, 0.8315256688419311], [0.38671475743664274, 0.35991105985971295], [0.20314636938840902, 0.286288084981262], [0.8177283154833807, 0.9037957573514619], [0.6220895584617758, 0.9472131875189801], [0.6799265866121214, 0.9321181979948256], [0.3045439017260675, 0.19215839921060252], [0.07115302028265635, 0.6512055966089577], [0.6306815055082894, 0.38448915581044973], [0.4199766023690835, 0.597294954027516], [0.4157553558523783, 0.5797561858110111], [0.4557890672181196, 0.18832068043989858], [0.5162719295759377, 0.6255123355948556], [0.09606531749088132, 0.9119618129143237], [0.2618264134157844, 0.1365229334895991], [0.8929710102510462, 0.3631863173506189], [0.013047000954481636, 0.6286494826309673], [0.010372281730892885, 0.7005437852850016], [0.7932744062738613, 0.472344932208724], [0.5361268285321439, 0.9530651747269369], [0.5484809904501707, 0.6154306014017084], [0.6231543617104888, 0.06997208080456752], [0.6121609671565241, 0.7841156114813835], [0.5911027959089393, 0.8451872156921644], [0.5267483865920456, 0.6972154105079387], [0.48632672871722016, 0.9278031697059202], [0.48757126147550967, 0.02917020188055819], [0.64881810361986, 0.13610801713854692], [0.6976336485626793, 0.1722421381028807], [0.6270133627239115, 0.4140388223131629], [0.41123392167712014, 0.32004681229002974], [0.31576170942907333, 0.9204872870141118], [0.32350934823125743, 0.7581503617769257], [0.9476719340875271, 0.7943407922051136], [0.4557811790189269, 0.6524064279082991], [0.791646898056853, 0.1519727675502055]]

In [13]:
city=list(range(len(points)))
coordinates={} ##dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)} 
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') 
for i,j in vars.keys():
    vars[j,i]=vars[i,j]
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city)
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 40 rows, 780 columns and 1560 nonzeros
Model fingerprint: 0x1bc0f0b7
Variable types: 0 continuous, 780 integer (780 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 40 rows, 780 columns, 1560 nonzeros
Variable types: 0 continuous, 780 integer (780 binary)

Root relaxation: objective 4.552090e+00, 59 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.85255    0    6          -    4.85255      -     -    0s
     0     0    4.88264    0    6          -    4.88264      -     -    0s
H    0   

[0,
 14,
 13,
 38,
 16,
 24,
 28,
 26,
 27,
 29,
 23,
 8,
 9,
 7,
 3,
 37,
 4,
 2,
 33,
 12,
 22,
 19,
 39,
 32,
 31,
 25,
 30,
 15,
 34,
 5,
 10,
 18,
 1,
 6,
 11,
 20,
 21,
 17,
 35,
 36]

In [14]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/60_0.py
points=[[0.47344832881322774, 0.8445066118262712], [0.42667851441093685, 0.1721258193904971], [0.6735424151232899, 0.887034705226131], [0.9650851797295693, 0.0884863024907192], [0.7051350745603394, 0.13273599013413828], [0.39322652472946484, 0.19733473538361213], [0.23157052753757934, 0.5055316071055826], [0.46419289013605836, 0.8639802645482697], [0.926220954668326, 0.04149120244492621], [0.6633211449954165, 0.19121971535688953], [0.652904527112747, 0.8777131113280576], [0.8179836717923227, 0.4239262990826118], [0.9110056570067554, 0.9378907880307196], [0.18655220682766482, 0.886977514101951], [0.1346899102882042, 0.16295541576972683], [0.5520794330550928, 0.7367594567886812], [0.24357697477884688, 0.5851103286522676], [0.475012174412216, 0.2613299009091472], [0.9613700557545213, 0.7412939610024412], [0.24101892912497846, 0.9960413425069854], [0.7351641209915455, 0.8727348954840347], [0.37929809077266463, 0.4419692702663044], [0.05623399215449587, 0.10305510104481364], [0.15408362678291654, 0.3000343204962157], [0.553457817562904, 0.4216068903829596], [0.3894029505176504, 0.3627482037013897], [0.9936002553311188, 0.011271651970295893], [0.7507631601153277, 0.14887620648606803], [0.7666788834777809, 0.9330206191327712], [0.43427653713795356, 0.5647365810363905], [0.4294597865467792, 0.21865134281150933], [0.7511111449838817, 0.6879194460518292], [0.5583016467626725, 0.8415908250805425], [0.21277122692022998, 0.29921399105389057], [0.6871902693502067, 0.2283340978732411], [0.8315928177046796, 0.4687083552899973], [0.6521691596543423, 0.6966954692825538], [0.49054399444992614, 0.48349486769260297], [0.1418113657050757, 0.4563650623678537], [0.6217608830782447, 0.09731154666249353], [0.5539846450566321, 0.02063892496307551], [0.031425216039552506, 0.3103477248420844], [0.46255615909257264, 0.734949101708845], [0.9635434100809315, 0.2166687959837592], [0.3113286466075157, 0.09330454646678465], [0.0331842406124323, 0.0641393773171558], [0.3147048779647671, 0.962162241989355], [0.1837292367427361, 0.9357284845259274], [0.010811347422066242, 0.5460513108230085], [0.17881635727438483, 0.35627119957658027], [0.3573162623791606, 0.10071993823818204], [0.26231238922475775, 0.3566918517021336], [0.3240233126842933, 0.37154684006360583], [0.4279047225185971, 0.2153141145311812], [0.3404536737745222, 0.8673691623769805], [0.19818541879373175, 0.742741291874901], [0.3037734228048268, 0.6012786204852401], [0.6931874565216026, 0.7431893127031736], [0.2672608522580998, 0.8808408331445726], [0.5127829587152501, 0.2923633684793193]]

In [15]:
city=list(range(len(points)))
coordinates={} ##dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)} 
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') 
for i,j in vars.keys():
    vars[j,i]=vars[i,j]
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city)
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 60 rows, 1770 columns and 3540 nonzeros
Model fingerprint: 0x579a53fa
Variable types: 0 continuous, 1770 integer (1770 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 60 rows, 1770 columns, 3540 nonzeros
Variable types: 0 continuous, 1770 integer (1770 binary)

Root relaxation: objective 5.868852e+00, 94 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.86885    0   12          -    5.86885      -     -    0s
     0     0    6.10107    0    6          -    6.10107      -     -    0s
   

[0,
 7,
 54,
 58,
 46,
 19,
 47,
 13,
 55,
 56,
 16,
 6,
 38,
 48,
 41,
 45,
 22,
 14,
 23,
 49,
 33,
 51,
 52,
 25,
 21,
 29,
 37,
 24,
 59,
 17,
 30,
 53,
 1,
 5,
 44,
 50,
 40,
 39,
 9,
 34,
 4,
 27,
 8,
 26,
 3,
 43,
 11,
 35,
 18,
 12,
 28,
 20,
 2,
 10,
 57,
 31,
 36,
 15,
 42,
 32]

In [16]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/60_1.py
points=[[0.8348646707961526, 0.4337418195416046], [0.2997104441187439, 0.317170653230346], [0.6083442500744562, 0.8106462422553058], [0.9996590567718706, 0.3559834981671559], [0.048887736610804255, 0.9059095077170982], [0.90124561637529, 0.5420010463426761], [0.6194860326597118, 0.9532758514814339], [0.4481857760646265, 0.42842042237232225], [0.26683761719038757, 0.7050561892242232], [0.7456851652010748, 0.775863326837873], [0.8944271843504544, 0.5098003031745066], [0.4504726456812925, 0.4549820198884694], [0.264923206712089, 0.7709369342769679], [0.38765925380453115, 0.988859642181622], [0.6357606962610997, 0.6850842827829484], [0.5370249329780846, 0.9436039452786049], [0.7797320877925895, 0.14381758757671592], [0.03791875683883594, 0.4341977577223466], [0.5196881856992968, 0.42326614662390083], [0.3074356434302282, 0.4102862658313471], [0.054177034374084165, 0.36562180402793876], [0.6148262744136557, 0.6830316030494369], [0.9456367872250259, 0.07800480898343931], [0.5686555465998295, 0.6768991476892677], [0.032763106276108234, 0.5334230399220969], [0.6027761427946036, 0.3958297093451717], [0.9082078007920638, 0.1283172191333456], [0.565115241364771, 0.265145366886516], [0.6201863322944623, 0.8710098313222686], [0.004879957647982325, 0.15181854330021838], [0.8344182426699203, 0.4184357662399927], [0.1221655452719349, 0.5873506793488918], [0.629401981388514, 0.0039045998959827966], [0.5997639446645008, 0.2477075209342693], [0.4572723489142503, 0.63702929435596], [0.5360202352860329, 0.6172290345616732], [0.8114891041874116, 0.12868981285013503], [0.6066160926558941, 0.968734837692049], [0.2791013651516835, 0.6719811169695226], [0.6242126532477423, 0.33526853413041235], [0.8477761996087197, 0.7849237114332853], [0.02867499378148408, 0.9498421612724652], [0.35821483350998107, 0.5806282875034364], [0.3258299673178944, 0.45175272281032686], [0.4738731324593558, 0.31677992691771595], [0.0937536745292723, 0.9337649806300677], [0.6445007904584887, 0.7021532189251039], [0.24153307767765375, 0.133542705190702], [0.8524509136951803, 0.01600179999871343], [0.19099870220900805, 0.7318895911678565], [0.14503032106028968, 0.8151899243387645], [0.13415270180328176, 0.7532367620337073], [0.433182553852543, 0.7393073397246894], [0.721725550772003, 0.6106643800824431], [0.6260866551895825, 0.7744965238582084], [0.7603320622416188, 0.533704334658873], [0.37860179956893825, 0.9938378072753422], [0.8037669808074359, 0.7710230955496441], [0.8801598080004537, 0.5070692029314755], [0.9162155077784558, 0.6196927284244128]]

In [17]:
city=list(range(len(points)))
coordinates={} ##dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)} 
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') 
for i,j in vars.keys():
    vars[j,i]=vars[i,j]
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city)
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 60 rows, 1770 columns and 3540 nonzeros
Model fingerprint: 0xd47ccf86
Variable types: 0 continuous, 1770 integer (1770 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 60 rows, 1770 columns, 3540 nonzeros
Variable types: 0 continuous, 1770 integer (1770 binary)

Root relaxation: objective 5.562190e+00, 89 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.56219    0   12          -    5.56219      -     -    0s
     0     0    5.88167    0    6          -    5.88167      -     -    0s
   

[0,
 30,
 3,
 26,
 22,
 48,
 36,
 16,
 32,
 33,
 27,
 44,
 7,
 11,
 18,
 25,
 39,
 55,
 53,
 46,
 14,
 21,
 23,
 35,
 34,
 52,
 42,
 43,
 19,
 1,
 47,
 29,
 20,
 17,
 24,
 31,
 38,
 8,
 12,
 49,
 51,
 50,
 4,
 41,
 45,
 56,
 13,
 15,
 37,
 6,
 28,
 2,
 54,
 9,
 57,
 40,
 59,
 5,
 10,
 58]

In [18]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/60_2.py
points=[[0.5460534102775446, 0.1456383256272228], [0.32361675301548465, 0.8297323833276045], [0.4379833665051287, 0.1419263763275722], [0.8225232848982362, 0.4486803835265315], [0.33300677606478857, 0.8927881290670375], [0.2961543399297183, 0.310441944815116], [0.867892406007475, 0.7355795330845499], [0.544229495616555, 0.18187290913426435], [0.8189119671075356, 0.1070139879973373], [0.5691233051156727, 0.15444421657904506], [0.9587720238886352, 0.5211733676342675], [0.162004849310583, 0.3394125975425639], [0.5161037994645787, 0.24830401440142413], [0.5616964819928226, 0.8879915608347805], [0.736123674652642, 0.15003037930859553], [0.49294003810930176, 0.4341482002337824], [0.9853681896540903, 0.36164178364746025], [0.640203538508434, 0.8179559330051736], [0.12007716207747554, 0.7392578553002586], [0.7554305314819015, 0.11198815532104156], [0.6586962987252869, 0.18086848291871316], [0.032339032802101264, 0.082943622848628], [0.7877451273966707, 0.498114098516774], [0.23051291198160884, 0.10605113987790538], [0.11305055814449216, 0.8444851909214219], [0.9630800015452522, 0.9184935547557525], [0.3648795849969595, 0.851378947149187], [0.2771084723517777, 0.6307014624511743], [0.9704603453282513, 0.354461829815085], [0.3368090776698206, 0.9321129541147445], [0.8557615371225127, 0.9283024458799148], [0.5837614627126918, 0.5009355032097622], [0.20376372251947783, 0.9261195191142869], [0.9710139761463433, 0.9790763216386945], [0.3504117884067185, 0.234152544617787], [0.29218128760190964, 0.6787178511087348], [0.8581212705981534, 0.873092197900814], [0.9656900921615228, 0.7297263318487438], [0.1281632447094856, 0.21741292136260293], [0.5920093420255224, 0.1503354644308612], [0.42525579650743695, 0.39196683276576216], [0.4967481658683631, 0.46850362693466974], [0.23501568918899618, 0.09315871357020789], [0.8139794887207009, 0.8814733595123084], [0.37464634047428913, 0.7902653998925686], [0.5309774869921917, 0.8696249950499269], [0.977228850868172, 0.23632649274041795], [0.6961983779307959, 0.8155194816225452], [0.5644605848353759, 0.20429168901303507], [0.9391141874088775, 0.7683432762331606], [0.39444019933032026, 0.8447546784756816], [0.20797295281636385, 0.5915304704995763], [0.4575016765153992, 0.03415164194319831], [0.634569977613942, 0.24582093221378787], [0.1383910584187351, 0.9585873847090403], [0.5131727381469425, 0.5000047458211035], [0.88933604077341, 0.5646851101209588], [0.91305911875185, 0.6123238515415169], [0.8527701140902684, 0.3428569587210816], [0.5869943286980466, 0.03782746963513206]]

In [19]:
city=list(range(len(points)))
coordinates={} ##dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)} 
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') 
for i,j in vars.keys():
    vars[j,i]=vars[i,j]
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city)
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 60 rows, 1770 columns and 3540 nonzeros
Model fingerprint: 0xc4fb1f1b
Variable types: 0 continuous, 1770 integer (1770 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 60 rows, 1770 columns, 3540 nonzeros
Variable types: 0 continuous, 1770 integer (1770 binary)

Root relaxation: objective 5.245671e+00, 89 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.24567    0   12          -    5.24567      -     -    0s
     0     0    5.35917    0    6          -    5.35917      -     -    0s
   

[0,
 7,
 48,
 12,
 53,
 20,
 14,
 19,
 8,
 46,
 16,
 28,
 58,
 3,
 22,
 56,
 10,
 57,
 6,
 37,
 49,
 25,
 33,
 30,
 36,
 43,
 47,
 17,
 13,
 45,
 50,
 26,
 44,
 1,
 4,
 29,
 32,
 54,
 24,
 18,
 51,
 27,
 35,
 31,
 55,
 41,
 15,
 40,
 5,
 11,
 38,
 21,
 42,
 23,
 34,
 2,
 52,
 59,
 39,
 9]

In [20]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/80_0.py
points=[[0.4775639022609286, 0.6601498049271367], [0.9886922195031573, 0.6307905288109501], [0.05929521009813399, 0.19876520151164867], [0.84296939686467, 0.602213680486809], [0.1543172997651986, 0.4836176895178711], [0.03378628256662364, 0.7611710471576867], [0.5483981559814484, 0.09489933674533124], [0.5176018786644723, 0.2868303733069677], [0.27688629660152786, 0.17197100766476558], [0.7900822846991936, 0.8929983992659012], [0.6999391804447114, 0.37420316437108236], [0.6652174222879, 0.2328235253865225], [0.6080364025413039, 0.8715421320080855], [0.6912830826379102, 0.7591004765231786], [0.19361414702343938, 0.23033420753439948], [0.8983693373020722, 0.03211738863723168], [0.4799238449870935, 0.6607860810924264], [0.24877033285471695, 0.25645391272533125], [0.8138932148262933, 0.23397437026412693], [0.08620024602424847, 0.5601815146150831], [0.5033668894579886, 0.1411925051119045], [0.8980847549838239, 0.7786531135535342], [0.6986576332082634, 0.20187688206090315], [0.8978726316453951, 0.5479503587142909], [0.9230775011612354, 0.6875358194322478], [0.9773662869121491, 0.5435674318498032], [0.5251943643614081, 0.7871376110844176], [0.3379457148005267, 0.8704780912220708], [0.14932843809578034, 0.3719905717022527], [0.9149997696039658, 0.7305805249163734], [0.36120838328831384, 0.765697645268481], [0.8101741397801695, 0.8431873720010302], [0.4192091060165951, 0.8713637988490983], [0.6763955900323813, 0.614373125422389], [0.12131903017837253, 0.3126556360986621], [0.47549122627742146, 0.33927020469369595], [0.5828238827866874, 0.29190929328947834], [0.3702714904122928, 0.30149122397894423], [0.8089626693301437, 0.3217218625137359], [0.7739790161968093, 0.5125715850454675], [0.9453867715793433, 0.43028076250920877], [0.3928388890805542, 0.13870173394557017], [0.21593203360894808, 0.46822032378747835], [0.5573885945390831, 0.07210340572795859], [0.6345849952247888, 0.08588498429174785], [0.296260674401157, 0.5665846601421924], [0.7294785133615664, 0.8059915395649406], [0.14078703589459307, 0.612522109470081], [0.12968140233792258, 0.15597155918175198], [0.8587794679524527, 0.0375697453454511], [0.21540437187290284, 0.0560155922523764], [0.5015641388889236, 0.29235482136404034], [0.09543113771650658, 0.536681033916273], [0.23160520128702122, 0.040165457819479666], [0.21284979854161046, 0.1913351382943722], [0.891519436793979, 0.10011915924862735], [0.9759233067192724, 0.4372693843975898], [0.7641389514538537, 0.06910646963971356], [0.8622261757206028, 0.9945130831261596], [0.8523772946602842, 0.3613781498387646], [0.5718502959293649, 0.7054001013614228], [0.901597445026436, 0.5747382571641898], [0.05493794685876052, 0.7411624928321829], [0.006141661940732179, 0.6876865673304564], [0.5510146806637641, 0.032183262723949446], [0.4677862295044478, 0.860380314465716], [0.3303605809081184, 0.7177862904356899], [0.06979451807016512, 0.3253859152725187], [0.5320445688636932, 0.23206180349083794], [0.7325408874709405, 0.04021610458999392], [0.07705519397569072, 0.10484483917338272], [0.6199601004341143, 0.8408677464063566], [0.053013045300391526, 0.8038234345668371], [0.571035925091339, 0.35450286620719895], [0.17178314126826755, 0.03522163216180474], [0.7779574559282301, 0.2211763903244689], [0.633816467230359, 0.8654346805268011], [0.57349213792291, 0.47747464665037564], [0.025677270128117824, 0.5948270009781111], [0.47720280635375834, 0.8312530428641747]]

In [21]:
city=list(range(len(points)))
coordinates={} ##dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)} 
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') 
for i,j in vars.keys():
    vars[j,i]=vars[i,j]
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city)
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 80 rows, 3160 columns and 6320 nonzeros
Model fingerprint: 0x90f8ec50
Variable types: 0 continuous, 3160 integer (3160 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 80 rows, 3160 columns, 6320 nonzeros
Variable types: 0 continuous, 3160 integer (3160 binary)

Root relaxation: objective 6.179992e+00, 118 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.17999    0    6          -    6.17999      -     -    0s
     0     0    6.47947    0   18          -    6.47947      -     -    0s
  

[0,
 16,
 60,
 33,
 39,
 10,
 38,
 59,
 40,
 56,
 25,
 23,
 61,
 3,
 1,
 24,
 29,
 21,
 58,
 9,
 31,
 46,
 13,
 71,
 76,
 12,
 26,
 79,
 65,
 32,
 27,
 30,
 66,
 45,
 47,
 62,
 72,
 5,
 63,
 78,
 19,
 52,
 4,
 42,
 28,
 34,
 67,
 2,
 48,
 70,
 74,
 50,
 53,
 41,
 8,
 54,
 14,
 17,
 37,
 35,
 51,
 7,
 68,
 20,
 6,
 43,
 64,
 44,
 69,
 57,
 49,
 15,
 55,
 18,
 75,
 22,
 11,
 36,
 73,
 77]

In [22]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/80_1.py
points=[[0.7837013760437266, 0.5148522566595909], [0.7964830110734875, 0.39258985752190323], [0.9148593302302198, 0.9788927570379207], [0.05175765997890003, 0.8396406997746803], [0.2704277170000684, 0.3882442605611419], [0.8552020302023721, 0.18722653768383113], [0.40497226940625053, 0.08355154844830315], [0.3561411703853311, 0.9073200073154527], [0.8933672816866863, 0.43776477142802916], [0.04378399008493994, 0.38534327816895575], [0.9607844176649761, 0.4144079162185772], [0.15971178871615177, 0.6955217169547566], [0.9294136912614616, 0.16097995633010753], [0.9420373482763141, 0.9361822945616598], [0.41304458880592787, 0.4372507131546176], [0.41090099929339197, 0.9239015483452189], [0.2532507039663551, 0.993844008921725], [0.7484190987695983, 0.7095015431800222], [0.8676917443263739, 0.5021837821467715], [0.7370486478559416, 0.8928746301146835], [0.6582893855612266, 0.35281395139693894], [0.27127997829722794, 0.6845149520575123], [0.049883304075435064, 0.7638603970977061], [0.5665543319552383, 0.248933472552901], [0.07347577857542709, 0.4721074642452302], [0.6571046859846552, 0.5840508415111436], [0.3228226023931744, 0.46118117856782526], [0.32292518000568304, 0.6800758378625018], [0.6899012793875303, 0.5701398406061626], [0.6870394581535122, 0.006450022459141391], [0.276649960273607, 0.7320828568566589], [0.37986100212699336, 0.38952441877898536], [0.011053989578304613, 0.624453681210333], [0.9493955597041998, 0.2612171298451803], [0.2990650471906192, 0.29480555968354816], [0.9306068352191323, 0.5647983165730992], [0.2677077074977162, 0.6752779108196147], [0.022276966699352196, 0.24505841010759866], [0.3462432979470089, 0.7315432581298402], [0.62131969982077, 0.6846874362353418], [0.37135314447496015, 0.48601290617747417], [0.4044919112909333, 0.5863025181166995], [0.9880007452377206, 0.012430062374893969], [0.5537029753316745, 0.7573189071052833], [0.7545880953793225, 0.8125121629482466], [0.5935384978048838, 0.23622527128242332], [0.9740108380922935, 0.6897367018254713], [0.6228711894236736, 0.6347070202240971], [0.6265610133419917, 0.4838834905321968], [0.9000587947081317, 0.9274860043982333], [0.8529297125774568, 0.38017113678791925], [0.22670057201614124, 0.5679608329241539], [0.7241575515395101, 0.2885560441639822], [0.45185788599378895, 0.8951669742150704], [0.7966182177315261, 0.986993631638664], [0.0931088022898594, 0.3681487128672758], [0.18720648851925892, 0.6107786725841489], [0.7619495342622649, 0.056068033303880105], [0.3738291639790222, 0.4923101920873124], [0.22956438275005675, 0.9411060254986704], [0.9907916954054569, 0.4790465192016998], [0.8619125630051966, 0.7578374800016301], [0.35814189120780904, 0.4978947308762387], [0.3350476711180431, 0.7358648076953608], [0.7965303778642984, 0.7200195852216011], [0.7308429112818263, 0.4047755411329472], [0.13961860392858094, 0.34315342515979586], [0.9885415058503702, 0.35540883544476354], [0.03751410971084057, 0.465295480260143], [0.7255970206342321, 0.01093048247688011], [0.3681843805929146, 0.14231946880472224], [0.031641312103702135, 0.053328314151356304], [0.3605753772219824, 0.6902587879741178], [0.29826735026823237, 0.599049242919924], [0.7674445619907824, 0.3748947941209283], [0.314062999434528, 0.6319106992933514], [0.7687032024008159, 0.43115767682586637], [0.8486438458944996, 0.6300329178639863], [0.7389231756407737, 0.993992545424655], [0.6562806127011681, 0.7617996801937678]]

In [23]:
city=list(range(len(points)))
coordinates={} ##dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)} 
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') 
for i,j in vars.keys():
    vars[j,i]=vars[i,j]
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city)
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 80 rows, 3160 columns and 6320 nonzeros
Model fingerprint: 0x93b1d0d7
Variable types: 0 continuous, 3160 integer (3160 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 80 rows, 3160 columns, 6320 nonzeros
Variable types: 0 continuous, 3160 integer (3160 binary)

Root relaxation: objective 7.020613e+00, 123 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    7.02061    0   12          -    7.02061      -     -    0s
     0     0    7.17534    0   14          -    7.17534      -     -    0s
  

[0,
 18,
 8,
 50,
 1,
 76,
 65,
 74,
 52,
 20,
 48,
 28,
 25,
 47,
 39,
 79,
 43,
 53,
 15,
 7,
 16,
 59,
 3,
 22,
 32,
 11,
 56,
 51,
 73,
 75,
 27,
 36,
 21,
 30,
 63,
 38,
 72,
 41,
 58,
 40,
 62,
 26,
 14,
 31,
 34,
 4,
 66,
 55,
 24,
 68,
 9,
 37,
 71,
 70,
 6,
 23,
 45,
 29,
 69,
 57,
 42,
 12,
 5,
 33,
 67,
 10,
 60,
 35,
 46,
 61,
 49,
 13,
 2,
 54,
 78,
 19,
 44,
 17,
 64,
 77]

In [24]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/80_2.py
points=[[0.5886369416906354, 0.013502511136143425], [0.13753743881018132, 0.4231837040544746], [0.08148838432018646, 0.8185815667454642], [0.8393370053610699, 0.6874223963280934], [0.9905114071786928, 0.6739590733181701], [0.39014475282826244, 0.10435535969414678], [0.8760628737454035, 0.28071808553051836], [0.37251463549591846, 0.5221380018730631], [0.681584527459118, 0.8281419656305039], [0.43044501344833386, 0.108421723139969], [0.21808530230892642, 0.007790656893033043], [0.7188154079706095, 0.6300039459450876], [0.31273183267787563, 0.8781727518803437], [0.479594717089338, 0.9525087984150802], [0.3499872091839761, 0.5076583164657519], [0.4073229755046003, 0.5613075667604488], [0.8789280072672032, 0.3368957750484042], [0.8661689350218021, 0.2137662523984274], [0.01895491299364649, 0.1306325365695693], [0.8399285681005842, 0.917049636994147], [0.9654251589869727, 0.08010675582042015], [0.7139228976194958, 0.1166413659010127], [0.3151905983394212, 0.5838345649614805], [0.24908606443309034, 0.1041625790532571], [0.500622152510579, 0.28105318957852754], [0.9971342798296706, 0.7248777230287564], [0.06737197056548161, 0.5949498224601801], [0.025042690050125604, 0.6707047440261151], [0.4762965080520448, 0.8280460872515722], [0.6862375054403844, 0.019138104511016052], [0.7139933342666336, 0.6389627721817478], [0.5463171310653445, 0.03489171420228809], [0.05739978345523333, 0.4083302748563613], [0.16977082771727092, 0.4899256784392586], [0.3893714430312214, 0.34508637713020507], [0.5801791489660546, 0.10055770120575747], [0.8468826617677193, 0.24968335552232357], [0.008707313290846841, 0.2910947255366775], [0.28322651196176873, 0.3744544164352429], [0.5099268345279185, 0.6944296021737016], [0.6704315239429962, 0.12059450544687556], [0.1813548851928677, 0.9527157642885258], [0.19177342437244904, 0.7581842840076015], [0.5868417777629691, 0.06519114105624557], [0.32917625165992104, 0.4994734996219079], [0.7063691186680346, 0.22199096441817312], [0.9584066600267764, 0.05414148038510791], [0.31893956221398545, 0.6047569829697025], [0.2676000313009995, 0.15862082035194625], [0.7206487074375659, 0.8899874074366722], [0.9872101275575715, 0.595008564050909], [0.06670756549298762, 0.6450101920526677], [0.3429203833302221, 0.5317594002721341], [0.6455941085441979, 0.6756427480317628], [0.16446759084856633, 0.23155305668896664], [0.8264168113790861, 0.8536773715191216], [0.46405715279753823, 0.6209301694178214], [0.5397397827396723, 0.4845095533886137], [0.46286493800089923, 0.74098331496336], [0.29099673379986946, 0.3814682134133237], [0.2177890376945033, 0.8554174279275958], [0.45457030513489016, 0.5745024870828223], [0.8554164990473138, 0.12323745919297313], [0.5355867650993613, 0.5986197145956675], [0.45776382140802374, 0.13992092864038808], [0.23181472615928667, 0.1575664604776006], [0.13892487228893835, 0.08845921533431844], [0.00131683021314144, 0.24078949303115926], [0.6201275229611357, 0.6501119918457303], [0.7173004924474048, 0.747940984585781], [0.8846151899328539, 0.8122225177353909], [0.033979726078961714, 0.9928081826081926], [0.6903268612469331, 0.4957612144136416], [0.8725023326297173, 0.7317756646515656], [0.13408693317415465, 0.004558383259129273], [0.223943000151684, 0.22503729255432559], [0.6446141119361338, 0.9684250238428089], [0.20916821849947875, 0.4534953832074127], [0.0539207240152666, 0.12955500603228431], [0.6313297522317804, 0.5641000597365997]]

In [25]:
city=list(range(len(points)))
coordinates={} ##dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)} 
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') 
for i,j in vars.keys():
    vars[j,i]=vars[i,j]
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city)
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 80 rows, 3160 columns and 6320 nonzeros
Model fingerprint: 0x6a4941a8
Variable types: 0 continuous, 3160 integer (3160 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 80 rows, 3160 columns, 6320 nonzeros
Variable types: 0 continuous, 3160 integer (3160 binary)

Root relaxation: objective 6.614417e+00, 117 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.61442    0   22          -    6.61442      -     -    0s
     0     0    6.93450    0   20          -    6.93450      -     -    0s
  

[0,
 29,
 40,
 21,
 45,
 62,
 46,
 20,
 17,
 36,
 6,
 16,
 50,
 4,
 25,
 3,
 73,
 70,
 55,
 19,
 49,
 76,
 8,
 69,
 30,
 11,
 53,
 68,
 79,
 72,
 57,
 24,
 34,
 38,
 59,
 44,
 14,
 7,
 52,
 22,
 47,
 15,
 61,
 56,
 63,
 39,
 58,
 28,
 13,
 12,
 60,
 41,
 71,
 2,
 42,
 27,
 51,
 26,
 33,
 77,
 1,
 32,
 37,
 67,
 18,
 78,
 66,
 74,
 10,
 23,
 65,
 54,
 75,
 48,
 5,
 9,
 64,
 35,
 43,
 31]

In [26]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/100_0.py
points=[[0.8186410516046002, 0.671023853262211], [0.2183801753833886, 0.01504553284700294], [0.4222879580871769, 0.5460926830505678], [0.8910788895697114, 0.7064594099265317], [0.369990709977302, 0.7651585689342871], [0.2386532664894112, 0.20281717397398835], [0.7612941988246403, 0.7087174971912249], [0.0595505392704061, 0.91530071023454], [0.1673004672866002, 0.13796794490185504], [0.21650319402866047, 0.9454367362554233], [0.27364140288169314, 0.30920228114376336], [0.34136778998855755, 0.21551618427637154], [0.6376969852788706, 0.4657225680718994], [0.9075211654920182, 0.8288034933211508], [0.6013982183131666, 0.9379410632711384], [0.3516880643985598, 0.10765163390115373], [0.8993053146970766, 0.8339993830118894], [0.4772242824836488, 0.35599141354594666], [0.8425422418190371, 0.30349372721582846], [0.5826180730886962, 0.20456846106043003], [0.03012411759490241, 0.8872302090386849], [0.5373001814832258, 0.2314569920316467], [0.3908828204594277, 0.1509859238859511], [0.6377151265398135, 0.23203756986439927], [0.8068778560413705, 0.5185745622274731], [0.8238804937173324, 0.7545505670806885], [0.8249093864923134, 0.0106225946180587], [0.24581432856555074, 0.9188956464439508], [0.4843630526237068, 0.8483179663117779], [0.02465761905388797, 0.4664971138929821], [0.7962063852254214, 0.1473071339798564], [0.41391345355068476, 0.400577216885457], [0.8487144064614759, 0.7102565436038364], [0.3183822686977036, 0.004675499528367211], [0.9470143547517333, 0.3835388997354713], [0.9426078394473189, 0.6665065476588964], [0.7455981053029204, 0.20139305156294018], [0.9200191593655932, 0.6431173120716583], [0.8082191015924146, 0.15864835248583697], [0.50189719169979, 0.26158179010929916], [0.8599294640973059, 0.14995662422964207], [0.39839356893912303, 0.08945683568306606], [0.6527839884376223, 0.29039681421509767], [0.7141933600156045, 0.9857480380287547], [0.6373011161168924, 0.9842538494244812], [0.2496790827399984, 0.7560376630076538], [0.7566337445061787, 0.8352444478824108], [0.702344197826351, 0.266234095555946], [0.9201597276826676, 0.3820236240770448], [0.918522511208706, 0.3191418831677626], [0.30758692513067176, 0.1756006324472975], [0.9779527872512176, 0.4944806964298294], [0.9508236953297554, 0.9307734457284723], [0.41780574038020135, 0.6807630949125231], [0.3326522822317032, 0.2604586986767885], [0.9912503804388063, 0.7428987290968617], [0.611343577948383, 0.011086319478697648], [0.2834095788797817, 0.4772624489603372], [0.20703428425007175, 0.9315139905981091], [0.10283828717330523, 0.6652258121634124], [0.04277167110455449, 0.00029014812553285196], [0.21400787165370538, 0.8194952376630799], [0.6254145126660395, 0.06512227234443169], [0.39228178175920847, 0.5770098607202805], [0.8851171001072751, 0.26458832560125034], [0.18148649436174746, 0.8933993973705538], [0.607277705462044, 0.31515690064780566], [0.7679053781097287, 0.10470509227932545], [0.8390411312767109, 0.40072148459151835], [0.8049518359082372, 0.5643434233757019], [0.003328792353874732, 0.3909937485864121], [0.3550717968332111, 0.6717431599149405], [0.4869088218363028, 0.8235880877977931], [0.8711770311164188, 0.7862412877587418], [0.44186490538020495, 0.4667310808369497], [0.25256176724317136, 0.346123007052054], [0.4659222646818153, 0.9756097302736881], [0.9688511269632721, 0.7716355503355089], [0.13339875760913922, 0.44959722603843966], [0.84618970726545, 0.808269268351917], [0.5228239659469139, 0.2759916237361978], [0.4967081503607699, 0.41324571928201437], [0.3907078859073686, 0.7202227731115383], [0.2549288865648347, 0.9695373303919774], [0.8626402185777182, 0.18015833524459224], [0.8722941200972311, 0.7188672902884162], [0.48521938840005363, 0.012689604467966298], [0.22290261610534523, 0.4878478403357861], [0.24110914650014947, 0.4669863346264109], [0.13665188322319866, 0.09181942428375844], [0.5111599048658537, 0.7083241080398237], [0.43232896955215305, 0.9268997853901731], [0.6866651641242613, 0.08833129094602699], [0.027420203466394732, 0.32660367165319915], [0.7930437766801576, 0.2419671804304523], [0.6662496291334401, 0.4772420285155592], [0.24890814542021467, 0.13843428198528673], [0.4512267928385575, 0.8027454767099166], [0.1990501493984539, 0.5356210395234792], [0.12952624109259647, 0.23038893516372205]]

In [27]:
city=list(range(len(points)))
coordinates={} ##dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)} 
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') 
for i,j in vars.keys():
    vars[j,i]=vars[i,j]
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city)
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 100 rows, 4950 columns and 9900 nonzeros
Model fingerprint: 0xfc47f7c6
Variable types: 0 continuous, 4950 integer (4950 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 100 rows, 4950 columns, 9900 nonzeros
Variable types: 0 continuous, 4950 integer (4950 binary)

Root relaxation: objective 7.751026e+00, 152 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    7.75103    0   22          -    7.75103      -     -    0s
     0     0    7.79249    0    6          -    7.79249      -     -    0s


[0,
 6,
 25,
 32,
 85,
 3,
 37,
 35,
 55,
 77,
 52,
 16,
 13,
 73,
 79,
 46,
 43,
 44,
 14,
 76,
 91,
 28,
 72,
 97,
 90,
 53,
 2,
 63,
 71,
 82,
 4,
 45,
 61,
 27,
 83,
 9,
 58,
 65,
 7,
 20,
 59,
 98,
 87,
 57,
 88,
 78,
 29,
 70,
 93,
 99,
 75,
 10,
 54,
 11,
 50,
 5,
 96,
 8,
 89,
 60,
 1,
 33,
 15,
 22,
 41,
 86,
 56,
 62,
 92,
 26,
 67,
 30,
 38,
 40,
 84,
 64,
 49,
 48,
 34,
 51,
 68,
 18,
 94,
 36,
 47,
 42,
 66,
 23,
 19,
 21,
 39,
 80,
 17,
 31,
 74,
 81,
 12,
 95,
 24,
 69]

In [28]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/100_1.py
points=[[0.8145457390131716, 0.7770650858559578], [0.959676348394301, 0.013920188596968974], [0.632405096175913, 0.7981632986072389], [0.8528751168776064, 0.38334762305317494], [0.7709959077133447, 0.1458149831106178], [0.9747179850814901, 0.6002986917286767], [0.6264324067249217, 0.3377803041685504], [0.9169460121355286, 0.6177263376078667], [0.8365401476508129, 0.9764085640285309], [0.14505243229741083, 0.5097687378105967], [0.45755153346381927, 0.03658364391451119], [0.0661884266366558, 0.6959328207446714], [0.09872403210521907, 0.673717877248846], [0.7236968977413872, 0.42315751610925356], [0.08989837508234733, 0.46248645515668263], [0.6020832396344695, 0.6207111232559746], [0.6099780263271879, 0.2558135153213005], [0.7048527985048962, 0.6123623278354984], [0.7608551610538915, 0.8609731780144757], [0.8569102849681196, 0.0659018438654917], [0.7487562822384543, 0.5739094881067228], [0.8989747211395617, 0.21221716405759916], [0.3168152868967312, 0.8429346979321398], [0.6093411644418874, 0.7292617434124172], [0.32714085395082737, 0.40329583684719794], [0.1529017852293798, 0.5731518065033988], [0.8428903931345942, 0.05128922068680308], [0.5654440643299518, 0.656012620910451], [0.316622780314311, 0.007142818825619179], [0.2241894931735967, 0.9412339988490577], [0.34362411622599376, 0.8974416044049367], [0.4869287121552507, 0.6016493244669405], [0.35461538018487593, 0.635143202344167], [0.015942006587028046, 0.6541980697514432], [0.8091914140680327, 0.8992938935936542], [0.1757209899079828, 0.8363695673243984], [0.7314175311455509, 0.5150902569878113], [0.9923835849614366, 0.6414323880319093], [0.9804783775006694, 0.45377731178970504], [0.5500823797045945, 0.9987986590905801], [0.6384213082708478, 0.9420009549210946], [0.01279782943358132, 0.21683931520227373], [0.5003104081455229, 0.15964262507287352], [0.31750581249611487, 0.9831610060995645], [0.5400469146993909, 0.9334391709161093], [0.020733468977962244, 0.5119156247360037], [0.8772248539130237, 0.3838502822892106], [0.35008845046219705, 0.1971207612318321], [0.6489392321452475, 0.9679975233085287], [0.6181196642568104, 0.7086327467520146], [0.38625051416849776, 0.5176751066230987], [0.29486584294408136, 0.3377173150545736], [0.23143206853821985, 0.10905350185802876], [0.8659669650686883, 0.8759842575202068], [0.23879607474916464, 0.0824660321948234], [0.4994665693361441, 0.062184383368498075], [0.8914326123902798, 0.5577330559516356], [0.9112079178010322, 0.5972442433833912], [0.1765673433131757, 0.6706256692783051], [0.43429261384363627, 0.2154371033177036], [0.21005538804530122, 0.046805942317310056], [0.9985730231524528, 0.6859685948389318], [0.3295357234414926, 0.2390448107262294], [0.8911299885895677, 0.22321449802826365], [0.7533258875121943, 0.38313644705004524], [0.705486704654983, 0.6064906882573944], [0.9020086607793112, 0.4417560272002923], [0.9734433675575128, 0.2794485540118321], [0.25033739354524465, 0.19443078853615925], [0.8189807800298327, 0.9544387950033978], [0.9540354051750761, 0.6349584569418019], [0.17466387378803883, 0.6932127981209193], [0.8640244158935326, 0.21405001717746164], [0.06602866035871313, 0.7125852881041216], [0.12035866829102804, 0.25895462824033566], [0.9762213833449718, 0.38111991948292], [0.47317310545486224, 0.8600946412562344], [0.48585532675477405, 0.2974368419290483], [0.445516798433628, 0.052050766602587784], [0.3824957022420412, 0.5717332588801973], [0.6654221881042757, 0.7904098799774728], [0.8057161901406308, 0.7899213271706582], [0.7264196595486583, 0.8682499072659982], [0.34046681446738347, 0.40474931989579965], [0.6041164315114075, 0.6305375068749752], [0.8381196214737316, 0.2180782517901363], [0.33427435790379656, 0.02849586018698158], [0.020779746601972238, 0.14451364838849445], [0.3641914673326134, 0.8828495274693723], [0.7277485091291489, 0.21998045594828164], [0.6003887123084433, 0.24534134563872945], [0.6263750201196441, 0.4915740242829958], [0.04814683571857137, 0.03114965374961476], [0.5100371793466927, 0.4841693871745464], [0.5956686954266219, 0.9826006704565636], [0.8816176957589108, 0.4419220810326203], [0.8668919402615586, 0.3970009197920784], [0.6621631904025926, 0.6336260068848977], [0.14008528336213855, 0.18423098433567087], [0.35734641145170176, 0.33780336634606867]]

In [29]:
city=list(range(len(points)))
coordinates={} ##dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)} 
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') 
for i,j in vars.keys():
    vars[j,i]=vars[i,j]
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city)
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 100 rows, 4950 columns and 9900 nonzeros
Model fingerprint: 0x40ff30f6
Variable types: 0 continuous, 4950 integer (4950 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 100 rows, 4950 columns, 9900 nonzeros
Variable types: 0 continuous, 4950 integer (4950 binary)

Root relaxation: objective 6.978058e+00, 150 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.97806    0   18          -    6.97806      -     -    0s
     0     0    7.17112    0   16          -    7.17112      -     -    0s


[0,
 53,
 8,
 69,
 34,
 18,
 82,
 40,
 48,
 94,
 39,
 44,
 76,
 88,
 22,
 30,
 43,
 29,
 35,
 71,
 58,
 12,
 73,
 11,
 33,
 45,
 14,
 9,
 25,
 32,
 79,
 50,
 83,
 24,
 51,
 99,
 62,
 47,
 68,
 98,
 74,
 41,
 87,
 92,
 60,
 52,
 54,
 28,
 86,
 78,
 10,
 55,
 42,
 59,
 77,
 6,
 16,
 90,
 89,
 4,
 26,
 19,
 1,
 85,
 72,
 63,
 21,
 67,
 75,
 38,
 5,
 37,
 61,
 70,
 7,
 57,
 56,
 66,
 95,
 96,
 46,
 3,
 64,
 13,
 36,
 20,
 65,
 17,
 97,
 84,
 15,
 91,
 93,
 31,
 27,
 49,
 23,
 2,
 80,
 81]

In [30]:
# %load C:/Users/mac/Desktop/Optimization/TSP data/100_2.py
points=[[0.4910054194915512, 0.023726959692338956], [0.14611261063128755, 0.4921832024055117], [0.7572537404044855, 0.8553493162727078], [0.40706409797946086, 0.5093253894997699], [0.026297550160891303, 0.5783589967945488], [0.3237316164472863, 0.9200259484804606], [0.702189723601015, 0.9590698514503693], [0.18887921273545927, 0.7121169208496607], [0.5719813267836703, 0.544995624433663], [0.34076859106892166, 0.19261077760180279], [0.8680588774976313, 0.42743171766141497], [0.36567339493986073, 0.7192328725757791], [0.5755862353144735, 0.4871390403275587], [0.2368114727364825, 0.11666312582309168], [0.49915005536955603, 0.3040925346539005], [0.9089549959025737, 0.9892402598638667], [0.9526163305721483, 0.005280773681558548], [0.09768104366222685, 0.6003784302929587], [0.4496318418727806, 0.5649691807130189], [0.5664018068252719, 0.7271640344968662], [0.9256139037839806, 0.31155743758295473], [0.7728937044902611, 0.2783512185674345], [0.6583922870341213, 0.5231308414932972], [0.39845476589455164, 0.13890654357082122], [0.9327669453757131, 0.3787988038178426], [0.0329761059219954, 0.09723548625266498], [0.8074809442971764, 0.6579047572766218], [0.41126847754891593, 0.137855170466462], [0.5756286531779666, 0.22858418132934066], [0.0560640209888541, 0.7656375914098702], [0.3113054228260289, 0.4755447323591787], [0.6144920634664274, 0.5368694507231542], [0.36629740826954393, 0.5938911933985184], [0.4960071292020237, 0.955043096988376], [0.10642557947346742, 0.5026408070471232], [0.1806174222621416, 0.06989452867477708], [0.2572303953342623, 0.6918097252427161], [0.9054439054619645, 0.5516872289212287], [0.4238883753544396, 0.6678306711952218], [0.8866300821609843, 0.4756785293339053], [0.005809856529586543, 0.8236709257146685], [0.04998214910753007, 0.2512816531343336], [0.4135923021607152, 0.3848719384783891], [0.1260293210626542, 0.39441036350924796], [0.8016793864853125, 0.40486392596668586], [0.09509090320878877, 0.9942940815606988], [0.6526565697686642, 0.5640090701171175], [0.5207810389986318, 0.4896831158183249], [0.21323885960819766, 0.44842667770345246], [0.40453975708335455, 0.23527544573727222], [0.5220031509924568, 0.46473451149207856], [0.987171910592569, 0.8804698928412691], [0.37192165433537205, 0.7404597229086419], [0.027404147020324454, 0.7826784235917019], [0.6530221643849828, 0.6433984414646284], [0.051675718181131036, 0.9063137379377386], [0.7332182976295775, 0.6866560995463314], [0.6995022461974523, 0.6611368072535938], [0.7864983993875719, 0.2584471918899355], [0.8367529829919188, 0.3611497079698489], [0.43695400869342527, 0.32079534608763305], [0.7285379259965483, 0.4350896258626443], [0.6050560740913516, 0.23763886604928564], [0.7765441132475467, 0.9281057151547134], [0.7542237533803272, 0.9805675771221243], [0.7174720604271909, 0.6012066089390091], [0.3693750995773508, 0.5167180618383734], [0.4377091392504997, 0.2791007079011849], [0.7633836916896187, 0.12237071771832764], [0.04251959469695177, 0.424472611530635], [0.7871452056396698, 0.4107876423461946], [0.886801484050027, 0.10358056377431124], [0.29500528580188856, 0.2080700346583424], [0.75074463720647, 0.017402012091496055], [0.19741029200060545, 0.5672361097787103], [0.24783120115658097, 0.9727728002455645], [0.17873121142029635, 0.5252527404100523], [0.41141294678256524, 0.6018660693038281], [0.8721047211168375, 0.6779064423796701], [0.46946430670073525, 0.5559743303152902], [0.45671435537854654, 0.8362479374563063], [0.892082242415828, 0.43152451550412285], [0.07618934417263312, 0.032058161668029994], [0.24787484050140118, 0.6430554885264641], [0.021226511795637948, 0.08524944436875914], [0.8941078937960171, 0.5318834548358585], [0.8929108804835114, 0.29468700491944777], [0.2067407371966271, 0.44680129054730744], [0.522998689756024, 0.12226614066427344], [0.9179208029513415, 0.901056976795306], [0.6220419237187508, 0.8227276977879184], [0.8601728389108588, 0.7768258271801857], [0.9207187994875454, 0.6729995173786438], [0.8008026841802066, 0.20947806568333982], [0.31047296432769067, 0.9909046869828022], [0.9145469745466228, 0.3754581999755935], [0.9119926762614473, 0.7364458555795356], [0.4576185931513047, 0.5787793719857944], [0.5980588573382708, 0.31742382178883743], [0.3209599534333357, 0.6970612101387755]]

In [31]:
city=list(range(len(points)))
coordinates={} ##dictionary
for i in city:
    coordinates[i]=(float(points[i][0]),float(points[i][1]))
dist={(c1,c2):distance(c1,c2) for c1,c2 in combinations(city,2)} 
m=gp.Model()
vars=m.addVars(dist.keys(),obj=dist,vtype=GRB.BINARY,name='x') 
for i,j in vars.keys():
    vars[j,i]=vars[i,j]
cons=m.addConstrs(vars.sum(c,'*')==2 for c in city)
m._vars=vars
m.Params.lazyConstraints=1
%time m.optimize(subtoureli)
vals=m.getAttr('x',vars)
selected=gp.tuplelist((i,j) for i,j in vals.keys() if vals[i,j]>0.5) 
tour=subtour(selected)
assert len(tour)==len(city) 
tour

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 100 rows, 4950 columns and 9900 nonzeros
Model fingerprint: 0xf5db9b1a
Variable types: 0 continuous, 4950 integer (4950 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 100 rows, 4950 columns, 9900 nonzeros
Variable types: 0 continuous, 4950 integer (4950 binary)

Root relaxation: objective 7.455474e+00, 133 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    7.45547    0    6          -    7.45547      -     -    0s
     0     0    7.59669    0   12          -    7.59669      -     -    0s


[0,
 27,
 23,
 49,
 9,
 72,
 13,
 35,
 82,
 84,
 25,
 41,
 43,
 69,
 34,
 4,
 17,
 74,
 76,
 1,
 87,
 48,
 30,
 66,
 3,
 79,
 18,
 97,
 77,
 32,
 38,
 52,
 11,
 99,
 36,
 83,
 7,
 29,
 53,
 40,
 55,
 45,
 75,
 94,
 5,
 33,
 80,
 19,
 90,
 2,
 63,
 6,
 64,
 15,
 89,
 51,
 91,
 96,
 92,
 78,
 26,
 56,
 57,
 54,
 65,
 37,
 85,
 39,
 10,
 81,
 95,
 24,
 20,
 86,
 71,
 16,
 73,
 68,
 93,
 58,
 21,
 59,
 44,
 70,
 61,
 22,
 46,
 31,
 8,
 12,
 47,
 50,
 42,
 60,
 67,
 14,
 98,
 62,
 28,
 88]